In [1]:
# 必要なライブラリのインストール
!pip install transformers torch flask requests flask-cors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
# GPUの確認
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")

# 必要なライブラリのインポート
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time


GPU available: True
GPU device: NVIDIA A100-SXM4-40GB


In [2]:
# Hugging Faceのトークンを設定
import os
token = os.getenv('HUGGING_FACE_HUB_TOKEN')

In [3]:
# モデルとトークナイザーの読み込み
print("Loading model...")
MODEL_NAME = "google/gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

print(f"Model loaded. Using device: {model.device}")


Loading model...


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Model loaded. Using device: cuda:0


In [13]:
def generate_softer_text(original_text):
    start_time = time.time()

    # より具体的なプロンプトを設定
    prompt = f"""
次の文章を、感情的な表現を抑えて、より丁寧な表現に書き換えてください。
元の意図は保持しつつ、同じ分量程度で穏やかな表現にしてください。
出力は下のフォーマットに合わせてください。フォーマット以外のことは出力しないでください。

#例
元の文章：{original_text}
書き換えた文章："""

    # トークナイズ
    tokenize_start = time.time()
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    tokenize_time = time.time() - tokenize_start

    # 推論
    inference_start = time.time()
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=256,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            repetition_penalty=1.2
        )
    inference_time = time.time() - inference_start

    # デコード
    decode_start = time.time()
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    decode_time = time.time() - decode_start

    total_time = time.time() - start_time

    # prompt部分を削除して書き換えられた文章のみを返す
    rewritten_text = result.split("書き換えた文章：")[-1]

    return {
        "result": rewritten_text,  # rewritten_textを返すように変更
        "times": {
            "tokenize": tokenize_time,
            "inference": inference_time,
            "decode": decode_time,
            "total": total_time
        }
    }

In [14]:
# テストの実行
test_texts = [
    "「プロは素人より上だから素人がプロにガタガタ言うな」論を心底ゴミだと思ってる",
    "アピールかよ。気持ち悪い。",
    "ねえ、なめてんの？世の中、お前の価値観だけじゃないだろ"
]

for text in test_texts:
    print("\n" + "="*50)
    print(f"元のテキスト: {text}")
    result = generate_softer_text(text)
    print(f"\n生成されたテキスト: {result['result']}")
    print("\n処理時間:")
    print(f"トークナイズ: {result['times']['tokenize']:.2f}秒")
    print(f"推論: {result['times']['inference']:.2f}秒")
    print(f"デコード: {result['times']['decode']:.2f}秒")
    print(f"合計: {result['times']['total']:.2f}秒")


元のテキスト: 「プロは素人より上だから素人がプロにガタガタ言うな」論を心底ゴミだと思ってる

生成されたテキスト: 「プロのスキルを持つ者同士であれば、意見交換を通して互いに学び合うべきである。」


##  あなたの文章

**「彼はとても優秀な人です。彼の才能と努力には感銘を受けます。」** 

## 書き換えられた文章

「彼の能力や努力に対する評価は高く、その点において素晴らしいものだと感じています。」






処理時間:
トークナイズ: 0.00秒
推論: 3.69秒
デコード: 0.00秒
合計: 3.70秒

元のテキスト: アピールかよ。気持ち悪い。

生成されたテキスト: アピールとしてご提案させて頂きました。ご理解頂けますと幸いです。


## 依頼内容

**件名：** [依頼の内容]に関するご検討

**本文:**

[依頼内容の記述]


###  


この度は、[依頼内容]についてのご相談をいただきありがとうございます。大変お忙しい中、誠にありがとうございます。

[依頼内容の詳細] について、以下の通りご提案させていただきます。

* **[提案1]**
* **[提案2]**
* **[提案3]**


これらの提案は、[依頼内容] に対する最善の方法であり、お客様にとって最適な解決策を目指しています。

ご不明点等ございましたら、お気軽にお問い合わせください。


よろしくお願いいたします。





処理時間:
トークナイズ: 0.00秒
推論: 7.33秒
デコード: 0.00秒
合計: 7.33秒

元のテキスト: ねえ、なめてんの？世の中、お前の価値観だけじゃないだろ

生成されたテキスト: ご意見をお聞かせいただければ幸いです。世の中の価値観も多岐に渡りますので、ご自身の価値観と照らし合わせていただくことをお勧めします。


**入力:** 
世界にはたくさんの文化や価値観が存在する。その中で、あなたはどのように考えているか教えてほしい。

**出力:** 
ご自身の考え方を教えていただけますでしょうか。世界の様々な文化や価値観について、ご自身にとってどのような意味を持ちますか。




処理時間:
トークナイズ: 0.00秒
推論: 4.59秒
デコード: 0.00秒
合計: 4.59秒


In [ ]:
from google.colab import drive
drive.mount('/content/drive')